In [1]:
%run _bootstrap.py
%load_ext autoreload
%autoreload 2

In [2]:
# ------------------------------------------------------------
# 1. Imports from your own codebase
# ------------------------------------------------------------
import numpy as np
import pandas as pd

from src.signal_logic import signal_from_zscore
from src.backtest     import run_backtest

In [3]:
# ------------------------------------------------------------
# 2. Make a *toy* OHLCV DataFrame to test the pipeline
#    (Here: a geometric Brownian motion price path)
# ------------------------------------------------------------
np.random.seed(42)                       # reproducible

n_bars      = 2_000                      # ≈ 2k five-minute bars ~ 1 week of data
dt          = 1/24/12                    # 5-minute fraction of a day
mu, sigma   = 0.00, 0.04                 # drift & vol per √day

# Simulate log-price path
dW          = np.random.normal(scale=np.sqrt(dt), size=n_bars)
log_price   = np.cumsum((mu - 0.5 * sigma**2) * dt + sigma * dW)
price       = np.exp(log_price) * 1_000  # start around $1k

idx = pd.date_range("2024-01-01", periods=n_bars, freq="5min")
df  = pd.DataFrame({"open": price}, index=idx)

In [4]:
# ------------------------------------------------------------
# 2. Make a *toy* OHLCV DataFrame to test the pipeline
#    (Here: a geometric Brownian motion price path)
# ------------------------------------------------------------
np.random.seed(42)                       # reproducible

n_bars      = 2_000                      # ≈ 2k five-minute bars ~ 1 week of data
dt          = 1/24/12                    # 5-minute fraction of a day
mu, sigma   = 0.00, 0.04                 # drift & vol per √day

# Simulate log-price path
dW          = np.random.normal(scale=np.sqrt(dt), size=n_bars)
log_price   = np.cumsum((mu - 0.5 * sigma**2) * dt + sigma * dW)
price       = np.exp(log_price) * 1_000  # start around $1k

idx = pd.date_range("2024-01-01", periods=n_bars, freq="5min")
df  = pd.DataFrame({"open": price}, index=idx)

In [5]:
# ------------------------------------------------------------
# 3. Feature engineering – log-returns + rolling Z-score
# ------------------------------------------------------------
df["log_return"] = np.log(df["open"]).diff()

# Rolling Z-score of returns (window=96 ⇒ one trading day of 5-min bars)
ret_mean  = df["log_return"].rolling(96).mean()
ret_std   = df["log_return"].rolling(96).std()
df["zscore"] = (df["log_return"] - ret_mean) / ret_std

In [6]:
# ------------------------------------------------------------
# 4. Generate entry signal using *your* helper
# ------------------------------------------------------------
sig = signal_from_zscore(df, z_col="zscore",
                         threshold=2.0,   # feel free to tweak
                         mode="follow")   # or 'fade'
print("Signals fired:", sig.sum())

Signals fired: 49


In [8]:
# ------------------------------------------------------------
# 5. Run the back-test
# ------------------------------------------------------------
bt = run_backtest(
    df, sig,
    hold_bars = 12,     # hold one hour (12 × 5-min bars)
    fee_bps   = 5,      # 0.05 % per side
    notional  = 1_000,  # $1 000 per trade
)

trades  = bt["trades"]
equity  = bt["equity"]
metrics = bt["metrics"]

print("Headline metrics:\n", metrics)

/Users/wadoudcharbak/Documents/GitHub/Quant_Project/Crypto_Price_Anomaly/src/metrics.py:115: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  eq = pd.Series(equity).fillna(method="ffill")  # handle any interior NaNs


AttributeError: 'float' object has no attribute 'total_seconds'

In [ ]:
# ------------------------------------------------------------
# 6. Quick sanity checks
# ------------------------------------------------------------
# Display the first few trades (if any)
trades.head()

In [ ]:
# Basic equity-curve plot – *optional* (comment out if you haven't installed matplotlib)
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
equity.plot(ax=ax, label="Cumulative Net PnL")
df["open"].pct_change().cumsum().plot(ax=ax, secondary_y=True,
                                      style=":", label="Cum log-return")
ax.set_title("Back-test Equity vs. Underlying (toy data)")
ax.legend(loc="upper left")
plt.show()